In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
df = df[['Message', 'Category']]

df = df.dropna()

# Розділення на вхідні дані (x) та мітки класів (y)
x = df['Message']
y = LabelEncoder().fit_transform(df['Category'])

# Розділення на тренувальну і тестову вибірки
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=47)

# Перетворення тексту в числові вектори за допомогою TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
x_train_vec = vectorizer.fit_transform(x_train)
x_test_vec = vectorizer.transform(x_test)

In [3]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.naive_bayes import MultinomialNB

#
nb_classifier = MultinomialNB()
nb_classifier.fit(x_train_vec, y_train)

# Прогнозування на тестовій вибірці
y_pred = nb_classifier.predict(x_test_vec)

# Оцінка точності та звіт по класифікації
accuracy = accuracy_score(y_test, y_pred)
print(f'Точність: {accuracy:.2f}')

# Детальний звіт
print(classification_report(y_test, y_pred, target_names=['Spam', 'Ham']))
print(confusion_matrix(y_test, y_pred))

Точність: 0.97
              precision    recall  f1-score   support

        Spam       0.97      1.00      0.98       961
         Ham       1.00      0.79      0.88       154

    accuracy                           0.97      1115
   macro avg       0.98      0.89      0.93      1115
weighted avg       0.97      0.97      0.97      1115

[[961   0]
 [ 33 121]]


### Проблема нульової ймовірності виникає, коли ми намагаємось класифікувати слово, яке не з'являлося у навчальних даних для певного класу (Ham/Spam).

In [5]:
# Ініціалізація наївного баєсівського класифікатора без Лапласового згладжування
nb_classifier_no_smoothing = MultinomialNB(alpha=0.0, force_alpha=True)  # Без згладжування
nb_classifier_no_smoothing.fit(x_train_vec, y_train)

# Ініціалізація наївного баєсівського класифікатора з Лапласовим згладжуванням
nb_classifier_with_smoothing = MultinomialNB(alpha=1.0)  # З згладжуванням
nb_classifier_with_smoothing.fit(x_train_vec, y_train)

# Нові коментарі для тестування
new_comments = [
    "You should revise the task that was given to you",
    "Free emails selling STRAIGHT UNDER YOUR WINDOW!",
]

# Векторизуємо нові коментарі
new_comments_vec = vectorizer.transform(new_comments)

# Отримуємо передбачення та ймовірності без згладжування
predictions_no_smoothing = nb_classifier_no_smoothing.predict(new_comments_vec)
probabilities_no_smoothing = nb_classifier_no_smoothing.predict_proba(new_comments_vec)

# Отримуємо передбачення та ймовірності із згладжуванням
predictions_with_smoothing = nb_classifier_with_smoothing.predict(new_comments_vec)
probabilities_with_smoothing = nb_classifier_with_smoothing.predict_proba(new_comments_vec)

# Виводимо результати
print("Результати без згладжування Лапласа:")
for comment, prediction, prob in zip(new_comments, predictions_no_smoothing, probabilities_no_smoothing):
    print(f"Коментар: {comment} \n   -> Результат фільтрації: {prediction} ({'Spam' if prediction == 1 else 'Ham'})")
    print(f"   -> Ймовірності: Ham: {prob[0]:.4f}, Spam: {prob[1]:.4f}\n")

print("Результати з згладжуванням Лапласа:")
for comment, prediction, prob in zip(new_comments, predictions_with_smoothing, probabilities_with_smoothing):
    print(f"Коментар: {comment} \n   -> Результат фільтрації: {prediction} ({'Spam' if prediction == 1 else 'Ham'})")
    print(f"   -> Ймовірності: Ham: {prob[0]:.4f}, Spam: {prob[1]:.4f}\n")

Результати без згладжування Лапласа:
Коментар: You should revise the task that was given to you 
   -> Результат фільтрації: 0 (Ham)
   -> Ймовірності: Ham: 1.0000, Spam: 0.0000

Коментар: Free emails selling STRAIGHT UNDER YOUR WINDOW! 
   -> Результат фільтрації: 0 (Ham)
   -> Ймовірності: Ham: 1.0000, Spam: 0.0000

Результати з згладжуванням Лапласа:
Коментар: You should revise the task that was given to you 
   -> Результат фільтрації: 0 (Ham)
   -> Ймовірності: Ham: 0.8514, Spam: 0.1486

Коментар: Free emails selling STRAIGHT UNDER YOUR WINDOW! 
   -> Результат фільтрації: 0 (Ham)
   -> Ймовірності: Ham: 0.8127, Spam: 0.1873



/home/yevhenii/.local/lib/python3.10/site-packages/sklearn/naive_bayes.py:897: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(


### При запуску коду без згладжування Лапласа ми можемо спостерігати, що модель класифікує деякі коментарі як "Ham" або "Spam" з ймовірностями, які можуть бути 0 для одного з класів. Це ілюструє проблему нульової ймовірності: якщо модель не бачила певне слово під час навчання, вона не може надати йому жодної ймовірності.

## Висновки
### Після виконання коду, результати для двох класифікаторів (згладжування і без) продемонструють різницю:

### Без згладжування деякі коментарі матимуть ймовірності 0 для одного з класів, що свідчить про проблему нульової ймовірності. А зі згладжуванням модель надає ненульові ймовірності, навіть якщо слова відсутні в навчальних даних, що дозволяє уникнути проблеми нульової ймовірності.